# Creation of the function to minimize
create the function $DN=DN_1+DN_2$\
With $$DN_1=\frac{\gamma}{2}(C(h)-h_n)^2$$ 
and $$DN_2=\frac{1}{2}(h-h_s+f(\alpha,x_c,x_a))^2$$
So $$DN=\frac{\gamma}{2}(C(h)-h_n)^2+\frac{1}{2}(h-h_s+f(\alpha,x_c,x_a))^2$$
Where 
$h = h(x_c, x_a)$ is the sought SSH, defined on the SWOT grid;\
$h_n = h_n(x_n)$ is the nadir SSH, defined on the nadir line, which coordinate is written $x_n$;\
$C$ interpolates the SWOT field onto the nadir line. $C(h)$ is the SWOT data as it would be seen by the nadir altimeter. $C(h)$ and $h_n$ are of the same nature (pixel size and locations) so that their difference can be computed and meaningful;\
$h_s = h_s(x_c, x_a)$ is the SWOT data (with their errors);\
$\alpha = (\alpha_0, \ldots, \alpha_6)$;\
$f$ is the error model, that should include all errors, with KaRIn errors;\
$\gamma$ is a weight to be prescribed depending upon the relative amplitude of the errors in nadir and SWOT data (a big value gives more weight to nadir data if their errors are small in comparison with SWOT, what is not the case...)\


## $DN_2$
$$DN_2=\frac{1}{2}(h-h_s+f(\alpha,x_c,x_a))^2$$

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jun  6 08:36:14 2020

@author: ASUS
"""


import netCDF4 as nc
import numpy as np
import os.path 
import scipy.optimize as so
import matplotlib.pylab as plt 
import pandas 
import copy 
from scipy import optimize
from scipy import linalg 
from scipy import spatial
from scipy.spatial.distance import cdist
from scipy.interpolate import griddata  
from scipy.interpolate import interp1d 
import calendar
from scipy import interpolate
import xarray as xr 

In [2]:
def obs_swotprojection(ssh_swot,flag_plot=False,boxsize = 99999):
    """
    NAME 
        obs_swotprojection

    DESCRIPTION 
        Produce and return the projection T defined in Metref et al. 2019.  
        
        Args:       
            ssh_swot (float array [nacross,nalong]): SWOT data,
            flag_plot (boolean): Flag if plots are produced,
            boxsize (int): Along track size of the sliding average box where the CER is performed
  
        Returns: 
            ssh_detrended (float array [nacross,nalong]): Projection T of SWOT, 
            aa1,bb1,cc1,ee11,ee12,ff11,ff12 (float array [nalong]): Regression coefficients 

    """      
   
    swot_fill_value = 2147483647. 
        
   
    # Mask invalid values 
    ssh_swotgrid=np.ma.masked_where(ssh_swot==swot_fill_value, ssh_swot,copy=False)   
    np.ma.masked_invalid(ssh_swotgrid) 
    
    # Plot SWOT observation
    if flag_plot:
            plt.figure()
            max_range=0.3 
            plt.imshow(ssh_swotgrid)  
            plt.colorbar(extend='both', fraction=0.042, pad=0.04)  
            plt.clim(-max_range,max_range) 
            plt.title('SWOT')
            plt.show()
          
    # Swath gap size (in gridpoints)
    n_gap = 10 
    
    # Calling obs_detrendswot
    ssh_detrended, aa1,bb1,cc1,ee11,ee12,ff11,ff12 = obs_detrendswot(ssh_swotgrid,n_gap,False,boxsize=boxsize)
    
    
    # Mask invalid values  
    np.ma.masked_invalid(ssh_detrended)  

          
        
        
    if flag_plot:
            plt.figure() 
            plt.imshow(ssh_detrended)  
            plt.colorbar(extend='both', fraction=0.042, pad=0.04)   
            plt.title('SWOT CER projection')
            plt.show()
            
            plt.figure()   
            plt.imshow(ssh_detrended-ssh_swotgrid)  
            plt.colorbar(extend='both', fraction=0.042, pad=0.04)   
            plt.title('SWOT CER projection - SWOT')
            plt.show()
          
         
    return ssh_detrended, aa1,bb1,cc1,ee11,ee12,ff11,ff12 

In [3]:
def obs_detrendswot(ssh_swotgrid0, n_gap, removealpha0=False,boxsize = 99999):
    """
    NAME 
        obs_detrendswot

    DESCRIPTION 
        Create the projection T (defined in Metref et al. 2019) of ssh_swotgrid0, an array of SSH in SWOT grid.
        
        Args:       
            ssh_swot (float array [nacross,nalong]): SWOT data,
            ngap (int): Size of the across track gap between the two SWOT swath,
            flag_plot (boolean): Flag if plots are produced,
            boxsize (int): Along track size of the sliding average box where the CER is performed
  
        Returns: 
            ssh_detrended0 (float array [nacross,nalong]): Projection T of SWOT, 
            aa1,bb1,cc1,ee11,ee12,ff11,ff12 (array[nalong]): Regression coefficients 

    """    
    # Size of averaging box for regression coefficient (only works if meanafter == True)
    #boxsize = 500 
    # Lenght of the swath
    dimtime = np.shape(ssh_swotgrid0)[0]
    # Width of the swath
    dimnc = np.shape(ssh_swotgrid0)[1]

    ssh_detrended0 = np.zeros_like(ssh_swotgrid0)
    ssh_detrended0[:,:] = ssh_swotgrid0[:,:]
    
    # Initialization of the regression coefficients
    a1=0
    b1=0 
    c1=0 
    e11=0
    e12=0 
    f11=0
    f12=0 
    
    
    # Initialization of the ssh_swot 
    h=np.zeros([dimtime,dimnc])
    
    
    # Three options: 
    # - typemean == 1: computing the regression coefficients (i_along times) on the sliding mean (ssh_acrosstrack)
    # - typemean == 2: averaging the regression coefficients computed at each ssh_acrosstrack (!! a lot slower !!)
    # - typemean == 3: computing the regression coefficients only once on the overall mean (ssh_acrosstrack)
    
    typemean = 2
    
    
    if typemean == 2 :  
        
        ssh_masked = np.ma.masked_where(ssh_swotgrid0>999.,ssh_swotgrid0) 
        ssh_masked = np.ma.masked_where(ssh_masked<-999.,ssh_masked) 
        aa1=np.zeros(dimtime) 
        bb1=np.zeros(dimtime) 
        cc1=np.zeros(dimtime) 
        ee11=np.zeros(dimtime)
        ee12=np.zeros(dimtime)
        ff11=np.zeros(dimtime)
        ff12=np.zeros(dimtime)
        i_along_init = 0
        i_along_fin = 1
        # Loop on the along track
        for i_along in range(dimtime): 
            # Average the ssh (sliding box)
            ssh_across = np.nanmean(ssh_masked[max(0,int(i_along-boxsize/2)):min(dimtime,int(i_along+boxsize/2)),:],0)  
            C=
            C=nadirpoints_in_swotgrid(lon_nadirpass,lat_nadirpass,ssh_nadir,lon_swotcut,lat_swotcut)
            
            if np.shape(ssh_across)[0]==dimnc:  
                if i_along_init == 0:
                    i_along_init = i_along
                nn = np.shape(ssh_across)[0] 
                if nn != 0: 
                    x_across = np.arange(nn)-int(nn/2) 
                x_across[x_across<0]=x_across[x_across<0]-n_gap/2
                x_across[x_across>=0]=x_across[x_across>=0]+n_gap/2
                i_along_fin = i_along

                # Cost function
                def linreg3(params): 
                    return np.sum( ( ssh_across-(params[0]+params[1]*x_across + params[2]*x_across**2 +np.append(params[3]+params[4]*x_across[x_across<=0],params[5]+params[6]*x_across[x_across>0],axis=0) ) )**2 ) 

                # Perform minimization 
                params = np.array([a1,b1,c1,e11,e12,f11,f12])
                coefopt = so.minimize(linreg3, params, method = "Powell") 
                aa1[i_along], bb1[i_along], cc1[i_along], ee11[i_along], ee12[i_along], ff11[i_along], ff12[i_along] = coefopt['x'][0], coefopt['x'][1], coefopt['x'][2], coefopt['x'][3], coefopt['x'][4], coefopt['x'][5], coefopt['x'][6] 
                a1=aa1[i_along]
                b1=bb1[i_along] 
                c1=cc1[i_along] 
                e11=ee11[i_along]
                e12=ee12[i_along]
                f11=ff11[i_along]
                f12=ff12[i_along]    
   
    
    ######################################################################
    # Make projection using the regression coefficients calculated above #
    ######################################################################
    for i_along in range(dimtime):  
        # Organize the coefficients for the projection 
        ssh_across = ssh_swotgrid0[i_along,ssh_swotgrid0[i_along,:]<999.]
        if typemean == 1 :
                # Average the coefficients (sliding box)
                if int(max(i_along_init,i_along-boxsize/2)) < int(min(i_along+boxsize/2,i_along_fin)):
                    a1 = np.mean(aa1[int(max(i_along_init,i_along-boxsize/2)):int(min(i_along+boxsize/2,i_along_fin))])
                    b1 = np.mean(bb1[int(max(i_along_init,i_along-boxsize/2)):int(min(i_along+boxsize/2,i_along_fin))]) 
                    c1 = np.mean(cc1[int(max(i_along_init,i_along-boxsize/2)):int(min(i_along+boxsize/2,i_along_fin))]) 
                    e11 = np.mean(ee11[int(max(i_along_init,i_along-boxsize/2)):int(min(i_along+boxsize/2,i_along_fin))])
                    e12 = np.mean(ee12[int(max(i_along_init,i_along-boxsize/2)):int(min(i_along+boxsize/2,i_along_fin))])
                    f11 = np.mean(ff11[int(max(i_along_init,i_along-boxsize/2)):int(min(i_along+boxsize/2,i_along_fin))])
                    f12 = np.mean(ff12[int(max(i_along_init,i_along-boxsize/2)):int(min(i_along+boxsize/2,i_along_fin))]) 
                else:
                    if i_along-boxsize/2 >= i_along_fin:  
                        a1 = np.mean(aa1[int(i_along_fin)])
                        b1 = np.mean(bb1[int(i_along_fin)]) 
                        c1 = np.mean(cc1[int(i_along_fin)]) 
                        e11 = np.mean(ee11[int(i_along_fin)])
                        e12 = np.mean(ee12[int(i_along_fin)])
                        f11 = np.mean(ff11[int(i_along_fin)])
                        f12 = np.mean(ff12[int(i_along_fin)])
                    if i_along_init >= i_along+boxsize/2:  
                        a1 = np.mean(aa1[int(i_along_init)])
                        b1 = np.mean(bb1[int(i_along_init)]) 
                        c1 = np.mean(cc1[int(i_along_init)]) 
                        e11 = np.mean(ee11[int(i_along_init)])
                        e12 = np.mean(ee12[int(i_along_init)])
                        f11 = np.mean(ff11[int(i_along_init)])
                        f12 = np.mean(ff12[int(i_along_init)])
                         
        elif typemean == 2: 
                a1 = aa1[i_along]
                b1 = bb1[i_along]
                c1 = cc1[i_along]
                e11 = ee11[i_along]
                e12 = ee12[i_along]
                f11 = ff11[i_along]
                f12 = ff12[i_along]  
                
        elif typemean == 3: 
                aa1=np.zeros(dimtime) + a1 
                bb1=np.zeros(dimtime) + b1 
                cc1=np.zeros(dimtime) + c1 
                ee11=np.zeros(dimtime) + e11
                ee12=np.zeros(dimtime) + e12
                ff11=np.zeros(dimtime) + f11
                ff12=np.zeros(dimtime) + f12 
                
        # Make the projection         
        nn = np.shape(ssh_across)[0] 
        x_across = np.where(ssh_swotgrid0[i_along,:]<999.)[0]-int(np.shape(ssh_swotgrid0)[1]/2) 
        x_across[x_across<0]=x_across[x_across<0]-n_gap/2
        x_across[x_across>=0]=x_across[x_across>=0]+n_gap/2 
        x_across_sized = x_across[ssh_across<999.]  
        if removealpha0 :
                ssh_detrended0[i_along,ssh_swotgrid0[i_along,:]<999.] = ssh_across[ssh_across<999.] - (a1+b1*x_across_sized + c1*x_across_sized**2+np.append(e11+e12*x_across_sized[x_across_sized<=0],f11+f12*x_across_sized[x_across_sized>0],axis=0) ) 
        else :
                ssh_detrended0[i_along,ssh_swotgrid0[i_along,:]<999.] = ssh_across[ssh_across<999.] - (b1*x_across_sized + c1*x_across_sized**2+np.append(e11+e12*x_across_sized[x_across_sized<=0],f11+f12*x_across_sized[x_across_sized>0],axis=0) )  
            

    
    return  ssh_detrended0, aa1,bb1,cc1,ee11,ee12,ff11,ff12

In [4]:
###################################################
# Masking nadir grid points outside of SWOT grid  #
###################################################   
def nadirpoints_in_swotgrid(lon_nadirpass,lat_nadirpass,ssh_nadir,lon_swotcut,lat_swotcut):
    """
    NAME 
        nadirpoints_in_swotgrid

    DESCRIPTION 
        Check which gridpoint of (ssh_nadir,lon_nadirpass,lat_nadirpass) are in SWOT grid 
        following the notations of point_in_swath.pdf manuscript 
        and mask ssh_nadir to mask all other nadir gridpoints.
        
        Args:       
            lon_nadirpass (float array [n_nadir]): longitude of nadir,
            lat_nadirpass (float array [n_nadir]): latitude of nadir,
            ssh_nadir (float array [n_nadir]): ssh values of nadir,
            lon_swotcut (float array [n_along,n_across]): longitude of SWOT,
            lat_swotcut (float array [n_along,n_across]): latitude of SWOT
  
        Returns: 
            ssh_nadir (masked float array [n_nadir]): ssh values of nadir available on grid SWOT

    """      
    
     #################################################
    # Checking if nadir grid poits are in left swath #
     #################################################
        
    # Number of nadir gridpoints
    n_nadir = np.shape(ssh_nadir)[0]
    
    # Number of half swath gridpoints
    ngap = np.int(np.shape(lon_swotcut)[1]/2)-1
     
    n0 = 0         # Index of bottom left of left swat
    n1 = ngap      # Index of bottom right of left swat
    
    a=[lon_swotcut[-1,n0]-lon_swotcut[0,n0],lat_swotcut[-1,n0]-lat_swotcut[0,n0]] 
    AX=[lon_swotcut[-1,n0]-lon_nadirpass[:],lat_swotcut[-1,n0]-lat_nadirpass[:]] 
    aa = np.tile([lon_swotcut[-1,n0]-lon_swotcut[0,n0],lat_swotcut[-1,n0]-lat_swotcut[0,n0]], (n_nadir, 1)).transpose() 
    AAXX = np.tile(np.dot(a,AX)/np.dot(a,a),(2,1))  
    u = AX - np.multiply(AAXX,aa)    
    b=[lon_swotcut[-1,n1]-lon_swotcut[0,n1],lat_swotcut[-1,n1]-lat_swotcut[0,n1]] 
    BX=[lon_swotcut[-1,n1]-lon_nadirpass[:],lat_swotcut[-1,n1]-lat_nadirpass[:]] 
    bb = np.tile([lon_swotcut[-1,n1]-lon_swotcut[0,n1],lat_swotcut[-1,n1]-lat_swotcut[0,n1]], (n_nadir, 1)).transpose() 
    BBXX = np.tile(np.dot(b,BX)/np.dot(b,b),(2,1))  
    v = BX - np.multiply(BBXX,bb) 

    a=[lon_swotcut[-1,n0]-lon_swotcut[0,n0],lat_swotcut[-1,n0]-lat_swotcut[0,n0]] 
    AX=[lon_swotcut[-1,n0]-lon_nadirpass[:],lat_swotcut[-1,n0]-lat_nadirpass[:]] 
    aa = np.tile([lon_swotcut[-1,n0]-lon_swotcut[0,n0],lat_swotcut[-1,n0]-lat_swotcut[0,n0]], (n_nadir, 1)).transpose() 
    AAXX = np.tile(np.dot(a,AX)/np.dot(a,a),(2,1))  
    u = AX - np.multiply(AAXX,aa)    
    b=[lon_swotcut[-1,n1]-lon_swotcut[0,n1],lat_swotcut[-1,n1]-lat_swotcut[0,n1]] 
    BX=[lon_swotcut[-1,n1]-lon_nadirpass[:],lat_swotcut[-1,n1]-lat_nadirpass[:]] 
    bb = np.tile([lon_swotcut[-1,n1]-lon_swotcut[0,n1],lat_swotcut[-1,n1]-lat_swotcut[0,n1]], (n_nadir, 1)).transpose() 
    BBXX = np.tile(np.dot(b,BX)/np.dot(b,b),(2,1))  
    v = BX - np.multiply(BBXX,bb) 

    in_first_swath = np.sum(u*v,axis=0)<0 

     ##################################################
    # Checking if nadir grid poits are in right swath #
     ##################################################
         
    n0 = ngap+1 # Index of bottom left of right swat
    n1 = np.int(np.shape(lon_swotcut)[1])-1 # Index of bottom right of right swat

    a=[lon_swotcut[-1,n0]-lon_swotcut[0,n0],lat_swotcut[-1,n0]-lat_swotcut[0,n0]] 
    AX=[lon_swotcut[-1,n0]-lon_nadirpass[:],lat_swotcut[-1,n0]-lat_nadirpass[:]] 
    aa = np.tile([lon_swotcut[-1,n0]-lon_swotcut[0,n0],lat_swotcut[-1,n0]-lat_swotcut[0,n0]], (n_nadir, 1)).transpose() 
    AAXX = np.tile(np.dot(a,AX)/np.dot(a,a),(2,1))  
    u = AX - np.multiply(AAXX,aa)    
    b=[lon_swotcut[-1,n1]-lon_swotcut[0,n1],lat_swotcut[-1,n1]-lat_swotcut[0,n1]] 
    BX=[lon_swotcut[-1,n1]-lon_nadirpass[:],lat_swotcut[-1,n1]-lat_nadirpass[:]] 
    bb = np.tile([lon_swotcut[-1,n1]-lon_swotcut[0,n1],lat_swotcut[-1,n1]-lat_swotcut[0,n1]], (n_nadir, 1)).transpose() 
    BBXX = np.tile(np.dot(b,BX)/np.dot(b,b),(2,1))  
    v = BX - np.multiply(BBXX,bb) 

    a=[lon_swotcut[-1,n0]-lon_swotcut[0,n0],lat_swotcut[-1,n0]-lat_swotcut[0,n0]] 
    AX=[lon_swotcut[-1,n0]-lon_nadirpass[:],lat_swotcut[-1,n0]-lat_nadirpass[:]] 
    aa = np.tile([lon_swotcut[-1,n0]-lon_swotcut[0,n0],lat_swotcut[-1,n0]-lat_swotcut[0,n0]], (n_nadir, 1)).transpose() 
    AAXX = np.tile(np.dot(a,AX)/np.dot(a,a),(2,1))  
    u = AX - np.multiply(AAXX,aa)    
    b=[lon_swotcut[-1,n1]-lon_swotcut[0,n1],lat_swotcut[-1,n1]-lat_swotcut[0,n1]] 
    BX=[lon_swotcut[-1,n1]-lon_nadirpass[:],lat_swotcut[-1,n1]-lat_nadirpass[:]] 
    bb = np.tile([lon_swotcut[-1,n1]-lon_swotcut[0,n1],lat_swotcut[-1,n1]-lat_swotcut[0,n1]], (n_nadir, 1)).transpose() 
    BBXX = np.tile(np.dot(b,BX)/np.dot(b,b),(2,1))  
    v = BX - np.multiply(BBXX,bb) 

    in_second_swath = np.sum(u*v,axis=0)<0  
     
    
    #####################################################################
    # Create boolean array true where nadir grid poits are in grid SWOT #
    #####################################################################
    in_pass = (in_first_swath+in_second_swath).astype(bool)
    
    ################################ 
    # Mask ssh_nadir appropriately #
    ################################ 
    ssh_nadir = np.ma.array(ssh_nadir, mask = np.invert(in_pass)) 
    
    return ssh_nadir

## Interpolation of SWOT on nadir

In [ ]:
# Creating interpolation function
swot2nadir_interp = interp2d(lon_swotpass[250:350], lat_swotpass[250:350], ssh_swotpass_truth[250:350])
#swot2nadir_interp

ssh_swotpass.shape

##### Interpolation on Nadir

swot_on_nadir_tmp = swot2nadir_interp(lon_nadirpass[110:120], lat_nadirpass[110:120])
print(swot_on_nadir_tmp.shape)

swot_on_nadir = np.diag(swot_on_nadir_tmp)

swot_on_nadir# SWOT interpolated on Nadir

### SSH(m) vs Nadir along track (x_al) for Nadir data
plt.figure()
plt.plot(lon_nadirpass[110:120],ssh_nadir[110:120])
plt.plot(lon_nadirpass[110:120],swot_on_nadir)
plt.ylabel('SSH (m)',fontsize=15)
plt.xlabel('lon (km)',fontsize=15)
plt.title('SSH (Nadir) along Nadir track', fontsize=15)
plt.show()